In [10]:
import binascii
with open('/sys/class/net/wpan0/address', 'r') as f :
    _strng = f.readline()
    _strng = b''.join(binascii.unhexlify(x) for x in _strng[:-1].split(':'))
    _sndBfr = b'\x41\xc8\x00\xff\xff\xff\xff'
    _sndBfr += _strng + b'\x06\x0d\x07\x03\x08\x01' + bytes(int('0')) + b'\x14\x01' + bytes(int('0')) + b'\x16\x01\xff'

In [ ]:
import socket
l2_sock = socket.socket(socket.AF_PACKET, socket.SOCK_RAW, socket.ntohs(0x0003))
l2_sock.bind(('wpan0', 0, socket.PACKET_BROADCAST))
l2_sock.setblocking(0)

In [ ]:
with open('/home/priyan/github-repo-offline/sdn-iot-ip-icn/hetnet-gw/logs/15_4-ccn/wpan2.log', 'r') as f : 
    for line in f.readlines() :
        if 'message' in line :
            frame = line[73:-3].encode('utf8')
            frame = frame.decode('unicode-escape')
            frame = [ x for x in frame ]
            print(frame, '\t', ord(frame[-4]))

In [ ]:
"29/12/2022 19:09:20-INFO-15_4-ccn.py-90 Total sent bytes 28 - message: ".__len__()

In [ ]:
print(_sndBfr)

In [ ]:
import socket
with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
    s.bind(('10.0.0.6', 65432))
    while True:
        data, addr = s.recvfrom(1024)
        print(f"Got {data} from {addr}")
        s.sendto(data, addr)


In [ ]:
a = 1
a.to_bytes(1, 'little')

In [ ]:
from select import select
from threading import Thread
from time import sleep
import socket
import threading

ndn154Sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
ndn154Sock.bind(('127.0.0.1', 65433))

it, jt = (0, 0)

def send(stop):
    global jt, it
    while True:
        jt = 0
        inputReady, e, o = select([ndn154Sock], [], [], 0.0)
        it += 1
        print(it, "\tInput ready")
        if stop():
            break
        #sleep(1)

def receive(stop):
    global jt, it
    while True:
        it = 0
        i, outptReddy, o = select([], [ndn154Sock], [], 0.0)
        jt += 1
        print(jt, "\tOutput ready")
        if stop():
            break
        #sleep(1)

stop_threads = False
Thread(target=send, args=(lambda : stop_threads, )).start()
Thread(target=receive, args=(lambda : stop_threads, )).start()
sleep(10)
stop_threads = True


In [ ]:
with open('/home/priyan/github-repo-offline/sdn-iot-ip-icn/hetnet-gw/logs/wifi-IP/readWriteSockTest.log', 'r') as f :
    _readCount, _writeCount = (0, 0)
    _indCycleReadCount, _indCycleWriteCount = (0, 0)
    _indCycleReadCounts, _indCycleWriteCounts = ([], [])
    _changeOver = False
    for line in f.readlines():
        if 'In' in line :
            _readCount += 1
            _changeOver = True
        else :
            _writeCount += 1
            _changeOver = False
        if _changeOver :
            if _indCycleWriteCount :
                _indCycleWriteCounts.append(_indCycleWriteCount)
                _indCycleWriteCount = 0
            _indCycleReadCount += 1
        else :
            if _indCycleReadCount :
                _indCycleReadCounts.append(_indCycleReadCount)
                _indCycleReadCount = 0
            _indCycleWriteCount += 1
    print("Total counts (read, write, read\%, write\%):",
    _readCount, _writeCount, 
    round(_readCount/(_readCount + _writeCount), 4) * 100, 
    round(_writeCount/(_readCount + _writeCount), 4) * 100)
    #print(_indCycleReadCounts, _indCycleWriteCounts)
    print("Maximum count per cycle (read, write):", max(_indCycleReadCounts), max(_indCycleWriteCounts))
    print("Minimum count per cycle (read, write):", min(_indCycleReadCounts), min(_indCycleWriteCounts))
    print("Mean count per cycle (read, write):", round(sum(_indCycleReadCounts)/_indCycleReadCounts.__len__(), 2), round(sum(_indCycleWriteCounts)/_indCycleWriteCounts.__len__(), 2))